In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [20]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub


from tensorflow.keras import layers

In [40]:
from keras.models import load_model


In [6]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [22]:
IMAGE_RES=224

In [75]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip=True)

In [76]:
#mymodel75 batchsize=32
#mymodel70 batchsize=5
#mymodel   batchsize=3
train_generator = train_datagen.flow_from_directory(
    directory=r"C:\Users\M.Plaban\Downloads\shelf_dataset-20191013T115924Z-001\shelf_dataset\train",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=3, 
    class_mode="categorical",
    shuffle=True,
    seed=65
)

Found 204 images belonging to 2 classes.


In [77]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [78]:
valid_generator = valid_datagen.flow_from_directory(
    directory=r"C:\Users\M.Plaban\Downloads\shelf_dataset-20191013T115924Z-001\shelf_dataset\val",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=3,
    class_mode="categorical",
    shuffle=True,
    seed=65
)

Found 40 images belonging to 2 classes.


In [23]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

W1013 17:48:39.903435  8824 deprecation.py:506] From C:\Users\M.Plaban\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [58]:
feature_extractor.trainable = False

In [79]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [80]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'])

In [81]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [83]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=15
)

Epoch 1/15
48/68 [====================>.........] - ETA: 3s - loss: 0.0173 - acc: 1.0000

KeyboardInterrupt: 

In [72]:
model.save('my_model.h5')